# Neo4J: Example of fictious bank clients

The following code templates have been accessed from: https://neo4j.com/docs/python-manual/current/

In [3]:
from neo4j import GraphDatabase
import pandas as pd

## Connect to the database

In [4]:
# --- Run in Codespaces: ---
URI = "bolt://neo4j:7687"

# --- Run locally: ---
# Install the required packages (requirements.txt)
# Start Docker Desktop and create an image, e.g. through Anaconda prompt:
# $ docker run -p7474:7474 -p7687:7687 -d -e NEO4J_AUTH=neo4j/secretgraph neo4j:latest
# To access your graph over Neo4J Desktop: Start Neo4J Desktop --> Graph Apps (left navigation) --> Neo4j Browser --> type in the credentials (neo4j/secretgraph) --> connect --> bolt://localhost:7687
# URI = "neo4j://localhost"

AUTH = ("neo4j", "secretgraph")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

## Writing manually into Neo4J

In [3]:
summary = driver.execute_query(
    "CREATE (:Person {name: $name})",
    name="Alice",
    database_="neo4j",
).summary
### summary appears two times, because '.summary' is a method that stores some metadata about the query execution. 
### The metadata includes the number of nodes created, the time it took to execute the query, and other information. 

print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))
### the '.format' method is used to format the output of the print statement. The curly braces {} are placeholders for the variables that are passed to the format() method.
### The variables are then inserted into the placeholders in the order they are passed to

###



/tmp/ipykernel_819/2632189726.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  summary = driver.execute_query(


Created 1 nodes in 1614 ms.


## Writing manually to the database in Neo4j Browser:

CREATE (:Person {name: 'Peter'})

## Querying

In [5]:
records, summary, keys = driver.execute_query(
    "MATCH (p:Person) RETURN p.name AS name",
    database_="neo4j",
)

# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

/tmp/ipykernel_18023/3877331262.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


{'name': 'Alice'}
The query `MATCH (p:Person) RETURN p.name AS name` returned 1 records in 315 ms.


## Update the database

In [6]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    SET p.age = $age
    """, name="Alice", age=42,
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

/tmp/ipykernel_18023/3817737189.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


Query counters: {'_contains_updates': True, 'properties_set': 1}.


In [7]:
records, summary, keys = driver.execute_query("""
    MATCH (alice:Person {name: $name})
    MATCH (bob:Person {name: $friend})
    CREATE (alice)-[:KNOWS]->(bob)
    """, name="Alice", friend="Bob",
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

Query counters: {}.


/tmp/ipykernel_18023/2454820279.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


## Delete from the database

In [8]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    DETACH DELETE p
    """, name="Alice",
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'nodes_deleted': 1}.


/tmp/ipykernel_18023/3368292491.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


## Query parameters

In [9]:
driver.execute_query(
    "MERGE (:Person {name: $name})",
    name="Alice", age=42,
    database_="neo4j",
)
### title is somewhat misleading...? The "merge" command is used to create a new node if it does not already exist. 
### If the node already exists, the command will not create a new node.


/tmp/ipykernel_18023/926086447.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  driver.execute_query(


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7c883f894bd0>, keys=[])

In [9]:
### is this an alternative way of doing the above? I can see how far out this could be better integrated into a loop
parameters = {
    "name": "Alice",
    "age": 42
}
driver.execute_query(
    "MERGE (:Person {name: $name})",
    parameters_=parameters,
    database_="neo4j",
)

C:\Users\pell\AppData\Local\Temp\ipykernel_32664\1909268578.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  driver.execute_query(


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000002B2F668A250>, keys=[])

## Writing a CSV file into Neo4J

In [12]:
# Connection details

# --- Run in Codespaces: ---
URI = "bolt://neo4j:7687" # Replace with your Neo4j bolt URI

# --- Run locally: ---
#URI = "neo4j://localhost"

username = "neo4j"  # Replace with your Neo4j username
password = "secretgraph"  # Replace with your Neo4j password

# Connect to the Neo4j database
driver = GraphDatabase.driver(URI, auth=(username, password))

def create_clients_and_transactions(tx, clients, transactions):
    # Create client nodes
    for client in clients['client_id']:
        tx.run("MERGE (c:Client {client_id: $client_id})", client_id=client)
    
    # Create transaction relationships (edges)
    for index, row in transactions.iterrows():
        tx.run(
            """
            MATCH (c1:Client {client_id: $client1}), (c2:Client {client_id: $client2})
            MERGE (c1)-[r:TRANSACTED_WITH {transactions: $transactions}]->(c2)
            """, 
            client1=row['client1'], 
            client2=row['client2'], 
            transactions=row['transactions']
        )

# Load CSV data into pandas DataFrame

# --- Run in Codespaces: ---
clients_df = pd.read_csv('workspace/example_data/swiss_bank_clients.csv')  # Update with correct path
transactions_df = pd.read_csv('workspace/example_data/swiss_bank_transactions.csv')  # Update with correct path

# --- Run locally: ---
# clients_df = pd.read_csv('example_data/swiss_bank_clients.csv')  # Update with correct path
# transactions_df = pd.read_csv('example_data/swiss_bank_transactions.csv')  # Update with correct path

# Load data into Neo4j
with driver.session() as session:
    session.write_transaction(create_clients_and_transactions, clients_df, transactions_df)

# Close the driver connection
driver.close()

print("Data loaded into Neo4j successfully!")


C:\Users\pell\AppData\Local\Temp\ipykernel_32664\3412886673.py:44: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_clients_and_transactions, clients_df, transactions_df)


Data loaded into Neo4j successfully!
